In [3]:
# !pip3 install --user torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip3 install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
# !pip3 install rich numpy matplotlib

# !pip3 install nms scikit-image

# !pip3 install jupyter-tensorboard

# !pip3 install "opencv-python-headless<4.3"

# !pip install ipywidgets
# !pip install nms

In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
import matplotlib.patches as patches
sys.path.append("..")

from Models.splinedist import *
import pytorch_lightning as pl
import torch
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor, RichProgressBar, RichModelSummary
from Datasets.CISD import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

logger = pl_loggers.TensorBoardLogger(
    save_dir=os.path.join(os.getcwd(), "../pl_logs_final2"),
    name="spline_dist/CISD",
    log_graph=True,
    default_hp_metric=True)

checkpoint_callback = ModelCheckpoint(
    monitor="loss_val",
    dirpath="Models_final21/weights",
    filename="splineDist-CISD8-{epoch:02d}-{loss_val:.2f}",
    save_top_k=2,
    mode="min",
)

earlystoping_callback = EarlyStopping("loss_val")

lr_logger = LearningRateMonitor(logging_interval="step")
summary = RichModelSummary(max_depth=-1)
progressbar = RichProgressBar()

callbacks = [progressbar, checkpoint_callback, lr_logger] 
device="cuda:1"


In [6]:
model = SplineDist(lambda1=0.9, lambda2=1e-4, learning_rate=1e-3, num_control_points=32, object_threshold=0.3, kernel_size=5, contourSize=512, nms_threshold=0.3)

In [7]:
# pl.seed_everything(47)
torch.cuda.set_device(device)
data = CISD_datamodule(contourSize=512)
# batch_x, batch_y = next(iter(data.val_dataloader()))
# showBatch((batch_x, batch_y))
# data = Nuclie_data(os.path.join(os.getcwd(), "src", "Datasets", "DSB18", "train"))
# for(img, target) in data:
#     print(img.shape)
#     # plt.imshow(img[0])
    

In [8]:
# from torchvision import io 
# for mask in os.listdir("C:\\SplineDist\\src\Datasets\\DSB18\\train\\00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552\\masks/"):
#     io.read_image(mask)

In [9]:
torch.cuda.empty_cache()
trainer = pl.Trainer(max_epochs=400,
                     gpus=[6],
                     callbacks=callbacks,
                     logger=logger,
                     num_sanity_val_steps=0)
trainer.fit(model, data)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


(device(type='cuda', index=6),) {}


/master/home/mbenimam/.local/lib/python3.8/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  if hasattr(mod, name):
/master/home/mbenimam/SplineDist/src/Experiments/../Models/unet.py:62: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
/master/home/mbenimam/SplineDist/src/Experiments/../Models/unet.py:63: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This resul

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃   ┃ Name                           ┃ Type         ┃ Params ┃      In sizes ┃    Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ 0 │ rgb2gray                       │ Conv2d       │      4 │             ? │            ? │
│ 1 │ segmentationBackbone           │ UNet         │ 17.3 M │   [1, 3, 256, │     [1, 128, │
│   │                                │              │        │          256] │    256, 256] │
│ 2 │ objectFinder                   │ Conv2d       │  3.2 K │ [1, 128, 256, │  [1, 1, 256, │
│   │                                │              │        │          256] │         256] │
│ 3 │ objectOverlap                  │ Conv2d       │  3.2 K │ [1, 128, 256, │  [1, 1, 256, │
│   │                                │              │        │          256] │         256] │
│ 4 │ ControlPointsAngleRegressor    │ Conv2d       │  102 K │ [1, 128, 256, │ [1, 32, 256, │
│   │                                │              │        │          256] │         256] │
│ 5 │ ControlPointsDistanceRegressor │ Conv2d       │  102 K │ [1, 128, 256, │ [1, 32, 256, │
│   │                                │              │        │          256] │         256] │
│ 6 │ cos                            │ Cos          │      0 │  [1, 32, 256, │ [1, 32, 256, │
│   │                                │              │        │          256] │         256] │
│ 7 │ sin                            │ Sin          │      0 │  [1, 32, 256, │ [1, 32, 256, │
│   │                                │              │        │          256] │         256] │
│ 8 │ bce                            │ BCEWithLogi… │      0 │             ? │            ? │
└───┴────────────────────────────────┴──────────────┴────────┴───────────────┴──────────────┘

Trainable params: 17.6 M                                                                     
Non-trainable params: 0                                                                      
Total params: 17.6 M                                                                         
Total estimated model params size (MB): 70

Output()

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /master/home/mbenimam/SplineDist/src/Experiments/Models_final21/weights exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


MemoryError: Unable to allocate 212. GiB for an array with shape (470466, 484760, 1) and data type uint8